In [ ]:
import sys
import json
from fabric import Connection
from invoke import Responder, UnexpectedExit

sys.version

In [ ]:
result = Connection('ubuntu@128.1.135.208').run('hostname', hide=False)

In [ ]:
result.stdout

In [ ]:
hosts = [
    "118.193.45.223",
    "128.1.135.208",
    "103.14.34.147",
    "23.91.97.32",
    "152.32.210.117",
    "23.91.98.101",
    "152.32.189.174",
    "36.255.220.243",
    "152.32.192.130",
    "101.36.117.63",
    "101.36.120.12",
    "101.36.122.83",
    "152.32.215.147",
    "152.32.251.236"
]

In [ ]:
# Check status

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run("curl -s http://localhost:1635/peers | jq '.peers | length'", hide=True)
    print(f'peers = {result.stdout.strip()}')
    print()

In [13]:
# Check peer balances

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run('cd swarm-single-machine && ./balances.sh')
    print()

==== 00/14 : 118.193.45.223 ====
swarm-00
{
  "balances": [
    {
      "peer": "1683a261ab1f95f0cd9c2d68a3de74b40196159ea88aeb597ffe8920b161ba92",
      "balance": 852000000000
    },
    {
      "peer": "16bcdb1a8bf1c19f540329932834b8a5479436cd7242218c0d3373564ce73b14",
      "balance": -21000000000
    },
    {
      "peer": "17118690786785a0f9c674264a802e1fead0da028af2850db65884265da95c1e",
      "balance": 20000000000
    },
    {
      "peer": "16056b5fce9350c9cd0e8ebed5224dd2d9ee406122ee9abaf17a344b666efe1e",
      "balance": 19000000000
    },
    {
      "peer": "160dac90d9362aed771d8177271491d65abcef82288e0338ed3dd126661bc52d",
      "balance": 290000000000
    },
    {
      "peer": "161ebc08f52ace17fdb5f37ae36ab85799bc98bc9709c8a01e4bf02b9935aabc",
      "balance": 38000000000
    },
    {
      "peer": "164e0cdbd00383028d0cae828ab84ca6f0481d7f610cf408c04e9a5662c53692",
      "balance": -32000000000
    },
    {
      "peer": "166174ee4d0cf1407b8ebc97e6ca18249f7654cc5715cd3

      "balance": -696000000000
    },
    {
      "peer": "166e1a25c66b32f271109ffb84e1717eb9aafdadd86fb646c8d21e85c784edcc",
      "balance": 27000000000
    },
    {
      "peer": "1702ce8c90e6d62b72961f314c012a202580bec066f42f4d6ac30c4f80b74f04",
      "balance": -21000000000
    },
    {
      "peer": "14f7f8a341915dba451ab3bf4e576abb38a38f7614d759cc54ab20e0b003e1fd",
      "balance": 280000000000
    },
    {
      "peer": "15acd39d9691b3c01d64acca82ef237c86f6f722149240c499e8ebfdda2a572d",
      "balance": 129000000000
    },
    {
      "peer": "164b0e527431bb6159ccc4a16487865ee6b2d7377981ce612a12e0638e9ee804",
      "balance": -25000000000
    },
    {
      "peer": "178306ee5979fc90c7f7588e363eb32d0193e7f24cbee5d363f68215da4d0835",
      "balance": -20000000000
    },
    {
      "peer": "149223af56c65dcaf7e774e30d9beeb058ae131fac0ce596d995942af1c171f0",
      "balance": 159000000000
    },
    {
      "peer": "165b4d56f46d18f19b5db97407267de3cab0ceda83665ef3fd92592c37b79fd7",


      "peer": "1644b37db82da84629729351b39d9124ec8615c8f49ead117096852364b66952",
      "balance": 86000000000
    },
    {
      "peer": "1667340b770efb9fa12dd9f0a9ff299caf5073816f7ca89846f5addec6b13149",
      "balance": -2000000000
    },
    {
      "peer": "13db9e31277cd19c7c677c4edd41ccfc724e6c0573cc59259901d5bd705c8a46",
      "balance": 224000000000
    },
    {
      "peer": "164f305088ce558e3479134582cf87647600e46af3e97b57d77e21437ba29d68",
      "balance": 224000000000
    },
    {
      "peer": "165779c18eee105046c69b86c5b1b27f66f01975def6bd5f242f7a8109d55dfe",
      "balance": 72000000000
    },
    {
      "peer": "165a9e516e38bb81d3304f4029d6d68b97cf284742872e7496b4af513961dd4a",
      "balance": -239000000000
    },
    {
      "peer": "167699625b651bf0c13553e6e4624b838ad301173dea87d11583b27e68a4b828",
      "balance": 12000000000
    },
    {
      "peer": "16a4d8d8de0e2325867a67483173529035e75548928ba101edda6bda8bd8771f",
      "balance": 16000000000
    },
    {
    

      "balance": 13000000000
    },
    {
      "peer": "5449cd70271245cdf02985f50fb1ae38e35e9e4f08ef38eb238dd04fe16dbacc",
      "balance": 23000000000
    },
    {
      "peer": "521a4da5469a19d5e5d2d5e8ed69c369d56b23355fc46dd0d0561f2164f86bb1",
      "balance": 18000000000
    },
    {
      "peer": "52297ca3750050cd846618f2766756f3c5d8e09b69ef067fa86ec1dee278bf33",
      "balance": -85000000000
    },
    {
      "peer": "522f1a4c092d76218960acf1b13cd415255fc2c961f3dee5d6e737c9115d1c68",
      "balance": 51000000000
    },
    {
      "peer": "522f390f50e2056cae92ef276af612e4e4722c54b8a925163f117d4ab1360ab6",
      "balance": -49000000000
    },
    {
      "peer": "52f494bb494a5ce19be7de97590a5740c47be42f05e943a1ce9e51105c9d6f5e",
      "balance": 22000000000
    },
    {
      "peer": "5200b314cc269d6f83ad2eb45e5bea967804475ef36a3701f8457e097502f221",
      "balance": 16000000000
    },
    {
      "peer": "527b7b645641af23d37c64583f90e2093d7d0bfcd396b35b0ba60d7cf830b4d1",
      

      "peer": "1243f80329afac0a3e7e93a6bf4990726ee0023fbb24d30cfec37cd790e45c5d",
      "balance": -25000000000
    },
    {
      "peer": "5087c7d323c667c9e18a00c3807a6d4eecf75b53796f9de0a21cc068fb733d27",
      "balance": 22000000000
    },
    {
      "peer": "517dd8a69888c07f2fec588276465b7f672dd5d4ed910de7bf3e058bfa3f7e28",
      "balance": -141000000000
    },
    {
      "peer": "5213ee0d704ba8b90fc7003b356bf8a99605b0f3181de8c6f608e228d0c657c9",
      "balance": -62000000000
    },
    {
      "peer": "527c51ed29ac63607481114b529049677245419714eaa13b7e2fcb4e553615d7",
      "balance": -18000000000
    },
    {
      "peer": "52b6a5b6ebbd78d58c49fef6d61f4c8df99bd26a203cbeac713e04dc827d5be1",
      "balance": 60000000000
    },
    {
      "peer": "51062afaadc5a06b74633eba5821790927ddbafd3aa3435a0176c6b31d9eee58",
      "balance": -21000000000
    },
    {
      "peer": "521c1bfd9241666ab9cddefb2ef499c6f1debced32a91652b00fb3a91a1e3eaa",
      "balance": -87000000000
    },
    {
 

...

      "balance": 18000000000
    },
    {
      "peer": "c42ab36a9e5943d3c1ff41533d2c4df7bbca94e3790c108e946dbe557b49eb81",
      "balance": 17000000000
    },
    {
      "peer": "c400908b910d58b5924784066d81b65ea3f338700d7b0de2ba2a9951853c2fec",
      "balance": -61000000000
    },
    {
      "peer": "c40098f565255ff3ea2fb4ee111b643b763c437226760995693b49df9c7aaac5",
      "balance": 72000000000
    },
    {
      "peer": "c4061ee1f370bb33844ce869681be14ef9e49e31c611153c8149e61be8100cf2",
      "balance": 106000000000
    },
    {
      "peer": "c416a6dfda74f7def5d570b6ca81c7e342cb354d456fa9b33acbd15797b522cf",
      "balance": -51000000000
    },
    {
      "peer": "c47ba90edde2a4848e1eb5f8b608df0c3a999698509c97dc6203ca9cbae17db6",
      "balance": -82000000000
    },
    {
      "peer": "c51d6231016f3b88a775aba36824d6645c0af55606d8c9c80b8d49a8759b2c1c",
      "balance": 23000000000
    },
    {
      "peer": "c403196cc34968edd362dd8469bdd236d391f7be9a32a14d0421507ef11d5e28",
    

      "peer": "c47cffe77a14c2194a087d1e93d30e787642057709a131e5419a740eaa9ca799",
      "balance": -17000000000
    },
    {
      "peer": "ba7428628330c102e5ff7ea756d153555df66afbcddbace0ca509357df2572a5",
      "balance": -22000000000
    },
    {
      "peer": "c66891a83b8b0148dbd16cafc3b4f4e8df0ae14e1026fe041ccdb91cef7e925e",
      "balance": 20000000000
    },
    {
      "peer": "d8e6aa62a8448413e989c017ba6ad5196676585fe78ef815005fc9f3289598e4",
      "balance": -113000000000
    },
    {
      "peer": "c41cce850ff7d8e0bf165c58f8eb1de84bb0c0bcc1aeb11a5b953105d912b2fa",
      "balance": -13000000000
    },
    {
      "peer": "c42d038cf1de30fb5fa77d436443985da3d17216bb18102c47c162894494ff97",
      "balance": 16000000000
    },
    {
      "peer": "c43f3944b60866709e3e2d04f6f75c8daf34875d07f0ed768801e47be0cd2f66",
      "balance": -30000000000
    },
    {
      "peer": "c4796ee29bbccb2dae1aeeec0cf1786b1e6d32a85d7250ceb29a4735c78371b0",
      "balance": -180000000000
    },
    {


      "balance": 1475000000000
    },
    {
      "peer": "21f334efec5b6a3ffeb306f1d23dcb98218a4497ee796a7c9a6b5ebb7c661181",
      "balance": -19000000000
    },
    {
      "peer": "2bfd512cb73ef89a00716f18d99f5261b5316fad3d4adc6001072951ca8d286e",
      "balance": -5923000000000
    },
    {
      "peer": "2443962bff7daba900db12635fd53892f578c5a27cccb4b219200cb3ca926063",
      "balance": -4135000000000
    },
    {
      "peer": "27993e336d61848670705ade8677d9276d3c4bae1be5f30404665098b9015531",
      "balance": -155000000000
    },
    {
      "peer": "25fdcab8628044c030961ccbe4b81ad9df05914ff3c7c069621e9e9f30272cda",
      "balance": -3786000000000
    },
    {
      "peer": "2a7be7a877e159db63f0ff6c284af7a957d2b1da82a1d437149abd2c779152c5",
      "balance": -23000000000
    },
    {
      "peer": "20aefee5aceec6f67da240d15ed4dacc9e0278642d1753f8c819fd5404b2c34c",
      "balance": -4000000000
    },
    {
      "peer": "22b16d7a9ff1d39238be239b9ec71964074de35048f11d0f6ae0cf4cf678

      "peer": "221490be53e9f9b59022b1d8bf2d39423ac497aee217a2213f58cc2217cabbef",
      "balance": 793000000000
    },
    {
      "peer": "27a55a66e23f6ce9b635eb541f7ba09c5751ba7fdd4024986b95414080e50404",
      "balance": -17000000000
    },
    {
      "peer": "21d9042344d5900d90c0bf76d328f3b4a3a369f74619067ae93f7f1dd22bab4e",
      "balance": 21000000000
    },
    {
      "peer": "232579771632dbc606d66af34e6f0f6586e852a836e8a153265b11500125a546",
      "balance": -236000000000
    },
    {
      "peer": "20a932284ad893ff9a335a4887d923aef26c085295395467cd742d49172a81dc",
      "balance": -3000000000
    },
    {
      "peer": "214068732601aec25844728867fffb981681e477bdc48ce668c5ddd7fa3180d0",
      "balance": 95000000000
    },
    {
      "peer": "210039794a8c2ead430c55bbf845b2241c553aa52afe0168838f73c689f0b511",
      "balance": -109000000000
    },
    {
      "peer": "373aea39e7b388d2dc4055cfc890da5421ea2b78304752f8c746e4bfcc84de86",
      "balance": -25000000000
    },
    {
 

      "balance": -234000000000
    },
    {
      "peer": "20cbaa53b16b1fc00244b485b5d9a30e4020e439c40a42564735fb58c9ca6ce9",
      "balance": -198000000000
    },
    {
      "peer": "2137b8df7369662ff2bc43db31209d0db9561bb5709e4cb6fd68b6e86d111e47",
      "balance": -425000000000
    },
    {
      "peer": "2145a3481f81120cc27e0b53f966a16bf135bc1e6f4905cb7bef5bf0fbec4263",
      "balance": -14000000000
    },
    {
      "peer": "21d8045c67e40c57d3c163718f804603710ac120fe27f4806172a2293e6f9132",
      "balance": -93000000000
    },
    {
      "peer": "23f0baf15f858f4fc4377460e1568b7a89a0d2a5b9f4e5ff438bd4e5f5804c91",
      "balance": 146000000000
    },
    {
      "peer": "28ccc9ccaa40b7747d3b037316db85e615b8ea39458836221b1c91f5c282799a",
      "balance": -8934000000000
    },
    {
      "peer": "20e5e4b52b4791cee38e5a93d174dd4d6b1f0e06907d9bb09652d4b588793691",
      "balance": 3059000000000
    },
    {
      "peer": "23de7029f70adf597b86f88d43623476677a7076872b479b236b55703b7e7

In [ ]:
# All down

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && ./down.sh')

In [ ]:
# All up

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && git pull')
    conn.run("cd swarm-single-machine && ./up.sh")

In [ ]:
# Rename hostname

sudopass = Responder(
    pattern=r'Password:',
    response='[please change]\n')

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run(f'hostnamectl set-hostname swarm-{i:02}', pty=True, watchers=[sudopass])
    conn.run('hostnamectl')
    print()

In [ ]:
# Add self hostname to /etc/hosts
# (to solve sudo error)

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('echo $(hostname -I | cut -d\  -f1) $(hostname) | sudo tee -a /etc/hosts')
    conn.run('cat /etc/hosts')

In [ ]:
# All addresses

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    conn.run('cd swarm-single-machine && ./addresses.sh')

In [ ]:
# Print all eth addresses

for _, host in enumerate(hosts):
    conn = Connection(f'ubuntu@{host}')
    result = conn.run('cd swarm-single-machine && ./addresses.sh', hide=True)
    payload = json.loads(result.stdout.strip())
    print(payload['ethereum'])

In [ ]:
# Try to init all

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    try:
        conn.run('git clone https://github.com/renning22/swarm-single-machine.git && cd swarm-single-machine && ./init.sh')
    except UnexpectedExit:
        print('Skip...')

In [ ]:
# Restart unhealthy

for i, host in enumerate(hosts):
    print(f'==== {i:02}/{len(hosts):02} : {host} ====')
    conn = Connection(f'ubuntu@{host}')
    conn.run('hostname')
    result = conn.run("curl -s http://localhost:1635/peers | jq '.peers | length'", hide=True)
    print(f'peers = {result.stdout.strip()}')
    if not result.stdout.strip():
        print('Restarting...')
        conn.run('cd swarm-single-machine && ./restart.sh')